$$
\newcommand{PDut}{\frac{\partial u}{\partial t}}
\newcommand{PDuS}{\frac{\partial u}{\partial S}}
\newcommand{PDuSS}{\frac{\partial ^2u}{\partial S^2}}
\newcommand{eps}{\varepsilon}
$$
$\newcommand{\a}{\alpha} \newcommand{\s}{\sigma} \newcommand{\half}{\frac{1}{2}} \newcommand{\F}{\mathcal{F}} \newcommand{\P}{\mathbb{P}} \newcommand{\par}{\partial} \newcommand{\R}{\mathbb{R}} \newcommand{\argmin}{\arg\!\min} \newcommand{\E}{\mathbb E} \newcommand{\lb}{\left [} \newcommand{\rb}{\right ]} \newcommand{\U}{\mathcal{U}}$


## 4. Deep learning based algorithm for BSDE
In this section, we will explore the implementation of a BSDE solver using deep learning structures. The algorithms are selected from [this paper](https://arxiv.org/pdf/1902.01599.pdf).

### DBDP1

The non-linear Feynman-Kac formula is stated as the following,

Given $t\in[0, T]$, 
* $(\Omega, \mathcal{F}, \{\F_t\}_{0\leq t \leq T}, \mathbb{P})$, the filtered probability space
* $\{X_s\}_{t\leq s \leq T}$ satisfies the forward SDE

$$
    dX_s = \mu(s,X_s) ds + \s(s,X_s) dW_s, \quad X_t = x.
$$
* $\{Y_s\}_{t\leq s \leq T}$ satisfies the backward SDE

$$
    dY_s =-f(s, X_s, Y_s, Z_s)ds + Z_sdW_s, \quad Y_T = g(X_T) \tag{1.1.1}
$$

* $(\mu, \s)$ and $(f, g)$ are the determinstic functions that satisfies the Lipschiz conditions and growth constraints

Suppose the (unique) function $u:[0, T] \to \R$ satisfies the following nonlinear PDE

$$  
\begin{align*}
   & \par_tu + \mathcal{L}u + f(t, x, u,  D_xu \s(t,x)) = 0 \tag{2.2.4} \\
   & u(T, x) = g(x) \tag{2.2.5}\\ 
\end{align*} 
$$

then the unique pair of $\F_t$-adapted process $(Y_s, Z_s)_{t \leq s \leq T}$ 

\begin{align*}
    Y_s &:= u(s, X_s)  \\
    Z_s &:= D_xu(s, X_s) \s(s,X_s)\\
\end{align*}

is the solution to BSDE $(1.1.1)$.

#### highlight of the algorithm
Suppose we have the same discretization of the FBSDE as in section 3.1. Now, recall that
\begin{align*}
    Y^P_{n} - Y^P_{n-1} &= -f(t_{n-1}, X^P_{n-1}, Y^P_{n-1},  Z^P_{n-1})\Delta t + Z^P_{n-1} \Delta W_{n} \tag{3.1.4} \\
\end{align*}
this means

\begin{align*}
    Y^P_{n} &= Y^P_{n-1} -f(t_{n-1}, X^P_{n-1}, Y^P_{n-1},  Z^P_{n-1})\Delta t + Z^P_{n-1} \Delta W_{n} \\
    &:= F(t_n, X^P_{n-1}, Y^P_{n-1}, Z^P_{n-1}, \Delta t, \Delta W_{n}) \tag{4.1.1}
\end{align*}

Now, consider a sequence of neural network approximation for $(Y^P_n, Z^P_n)_{n=0}^{N-1}$ as the following

\begin{align*}
    Y^P_{n} &= u(t_n, X^P_n) &\sim \mathscr{U}_{n}(X^P_n ; \zeta_n) \\
    Z^P_{n} &= D_xu(s, X_s) \s(s,X_s) &\sim \mathcal{Z}_{n}(X^P_n; \eta_n)
\end{align*}

where we define $\theta_n:=(\zeta_n, \eta_n)$ to be the set of parameters of the pair of neural network approximation $(\mathscr U_n, \mathcal Z_n)$ at time $t_n$.


Then, the equation $(4.1.1)$ becomes

\begin{align*}
    \mathscr{U}_n(t_n, X^P_n) &= F(t_n, X^P_{n-1}, \mathscr U_{n-1}, \mathcal Z_{n-1}, \Delta t, \Delta W_{n}) \tag{4.1.2}
\end{align*}

Equation $(4.1.2)$ is the key to the algorithm for fitting the above neutral network approximation iteratively backward, starting from $t_N=T$.



#### backward iteration

Here is the proposed scheme of DBDP1. We are aim to derive the optimal estimation $(\hat{\mathscr U}_n, \hat{\mathcal Z}_n)_{n=1}^N$ with the parameter $\{\theta^*_n\}_{n=1}^N = (\zeta^*_n, \eta^*_n)_{n=1}^N$ given the discretization scheme $P$.
* For $n = N$, define $\hat{\mathscr U}_N := g$.
* For $n = N-1,...,0$, we solve the optimization problem,

\begin{align*}
    \theta_n^* &= \argmin_{\theta} L_n(\theta) \\
    & = \argmin_{\theta} \frac{1}{M} \sum_{m=1}^M \left( \hat{\mathscr U}_n(X^P_{m, n}) - F(t_n, X^P_{m, n-1}, \mathscr U_{n-1} (\zeta), \mathcal Z_{n-1} (\eta), \Delta t, \Delta W_n) \right)^2
\end{align*}

Then, we store the optimal parameters $\theta^*_n$ for the model

$$
    \hat{\mathscr U}_n = \mathscr U_n(\cdot, \zeta^*_n) \quad \text{and} \quad \hat{\mathcal Z}_n = \mathcal Z_n(\cdot, \eta^*_n)
$$


Here, we immediately notice that the LHS of equation $(4.1.2)$ is known if we solve the following optimization  problem $(4.1.3)$ iteratively backward from $t_N=T$, where we are given $\mathscr U_N(T, X_T) := Y_T = g(X_T)$.